In [1]:
from fastembed import SparseTextEmbedding
from RAG.RAG_query import sparse_query_search
from RAG.RAG_query import dense_query_search
import os

from RAG.RAG_query import get_topics_list
from config.loadConfig import load_project_config
from DB.qDrant import check_qdrant_status, QdrantConfig
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from RAG.RAG_Retrieve import custom_embed_function

# Encode the query text using the same model

# Connect to Qdrant
config = load_project_config(os.path.join(os.getcwd(), "config/config.yaml"))
qdrantConfig = QdrantConfig(
    host=config["db"]["qdrant"]["connection"]["host"],
    port=config["db"]["qdrant"]["connection"]["port"],
    timeout=config["db"]["qdrant"]["connection"]["timeout"]
)
status, client = check_qdrant_status(qdrantConfig)
embedding_model = SentenceTransformer(config['embedding_model'])
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")
small_collection_name = config['db']['collections']['collection_small']['name']
large_collection_name = config['db']['collections']['collection_large']['name']

Connected. Collections: ['wiki_large_chunks', 'wiki_small_chunks']


In [2]:
get_topics_list(client, num_topics=100)

0002148b-bd70-425a-b567-df0d26086003 Feleti Sevele
0003d157-639b-4841-a514-ae4f53a7232b Abdulaziz
000472b5-a875-4fe4-9c14-199d372c4247 Bede
0006438e-760a-4bca-b77b-9603315bc238 Apollo 12
0006ac4b-bff1-4552-a8dc-b512a821153b Admiralty (United Kingdom)
0008d1b0-4812-49c0-b13a-004a221bdb4e Anglicanism
00091464-2664-41b8-b2df-76bb16e5d7c7 American Civil Liberties Union
000d2c92-16cc-4047-925f-25bb58f8d4c6 Aleister Crowley
000f09ab-f2f8-47a9-a0b9-e5308c88c9c9 History of Cameroon
00115b13-fbd7-4fea-b19b-21d48261deb5 Riaumont
00117c20-3066-4460-be67-3d3ea3eedf90 Calcium
0011a341-87ba-41f8-94b9-ff098cf47d95 Acts of Union 1707
0014a554-901d-43b9-9cf9-fbae06b1da78 Anatomy
001605af-fd2c-4b15-a931-f88cdf646d56 1985–86 in English football
0016aad5-da48-448f-934e-b8da94c3b130 American National Standards Institute
001735ea-8a83-4168-af9a-1815ce572bd5 Battlecruiser
00176d00-b342-4d7e-b867-66dbd3f8fab8 Burgundians
0017cd6b-376a-49a7-94e9-79315dd685c1 Battle of Stalingrad
001869d7-fefa-4f00-908c-2140d6c

In [3]:
query_text = "Who was Charlemagne?"
results = sparse_query_search(client, query_text, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
The Economist featured a weekly column entitled "Charlemagne", focusing generally on European affairs and, more usually and specifically, on the European Union and its politics.  Actor and singer Christopher Lee's symphonic metal concept album Charlemagne: By the Sword and the Cross and its heavy metal follow-up Charlemagne: The Omens of Death feature the events of Charlemagne's life.
of his father, Pepin. Desiderius sent his own ambassadors denying the pope's charges. The ambassadors met at Thionville, and Charlemagne upheld the pope's side. Charlemagne demanded what the pope had requested, but Desiderius swore never to comply. Charlemagne and his uncle Bernard crossed the Alps in 773 and chased the Lombards back to Pavia, which they then besieged. Charlemagne temporarily
In April 2014, on the occasion of the 1200th anniversary of Charlemagne's death, public art Mein Karl by Ottmar Hörl at Katschhof place was installed betwe

In [4]:
query_text = "Who was Charlemagne?"
results = dense_query_search(client, query_text, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
384
him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".
Visigoths and Carolingians: the legend of Charlemagne
Visigoths and Carolingians: the legend of Charlemagne
In July 2022, Charlemagne featured as a character in an episode of The Family Histories Podcast, and it references his role as an ancestor of all modern Europeans. He is portrayed here in later life, and is speaking Latin, which is translated by a device. He is returned to 9th Century Aquitaine by the end of the episode after a DNA sample has been extracted.  Notes  References  Citations
Jews in Charlemagne's realm


In [14]:
hierarchical_search(client, query_text, use_deep_embedding=True, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
384
Administration  Organisation The Carolingian king exercised the bannum, the right to rule and command. Under the Franks, it was a royal prerogative but could be delegated. He had supreme jurisdiction in judicial matters, made legislation, led the army, and protected both the Church and the poor. His administration was an attempt to organise the kingdom, church and nobility around him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".  Military Charlemagne's success rested primarily on novel siege technologies and excellent logistics rather than the long-claimed "cavalry revolution" led by Charles Martel in 730s. However, the stirrup, which made the "shock cavalry" lance charge possible, was not introduced to the Frankish kingdom until the late eighth century.  Horses were used extensi

[Record(id='90c224c1-60e5-42a8-9cd0-69faafa396cd', payload={'text': 'Administration\n\nOrganisation\nThe Carolingian king exercised the bannum, the right to rule and command. Under the Franks, it was a royal prerogative but could be delegated. He had supreme jurisdiction in judicial matters, made legislation, led the army, and protected both the Church and the poor. His administration was an attempt to organise the kingdom, church and nobility around him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".\n\nMilitary\nCharlemagne\'s success rested primarily on novel siege technologies and excellent logistics rather than the long-claimed "cavalry revolution" led by Charles Martel in 730s. However, the stirrup, which made the "shock cavalry" lance charge possible, was not introduced to the Frankish kingdom until the late eighth century.\n\nHorses were

In [5]:
result = client.retrieve(
    collection_name="wiki_large_chunks",
    ids=["035951e1-74e1-450e-a6a8-5bf9440fbad1"],
    with_payload=True
)
print(result)

[]
